In [98]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='5208762d-f69f-4fe8-803f-1c0172e4fc0c', project_access_token='p-ae7e139824883b7d37598fea06719b3d12f99449')
pc = project.project_context

In [99]:
#!conda install -c conda-forge geopandas --yes

In [100]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [101]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge geopandas --yes
import geopandas as gpd

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Reading London's GeoJson file and obtaining Neighberhood geographical locations

In [102]:
df0 = gpd.read_file(project.get_file('London.geojson'))
df0.rename(columns = {'name': 'Neighborhood'}, inplace = True)
df0.drop(['cartodb_id', 'created_at', 'updated_at'], axis = 1, inplace = True)

In [103]:
df0.head()

,Neighborhood,geometry
0,Barking and Dagenham,"MULTIPOLYGON (((0.07263 51.53122, 0.07263 51.5..."
1,Barnet,"MULTIPOLYGON (((-0.25216 51.58802, -0.25218 51..."
2,Bexley,"MULTIPOLYGON (((0.19886 51.45817, 0.19886 51.4..."
3,Brent,"MULTIPOLYGON (((-0.29050 51.53570, -0.29056 51..."
4,Bromley,"MULTIPOLYGON (((0.09379 51.32025, 0.09347 51.3..."


In [104]:
london = pd.DataFrame()

In [105]:
london['Neighborhood'] = df0.Neighborhood
london['Longitude'] = df0.centroid.x
london['Latitude'] = df0.centroid.y

In [106]:
london

,Neighborhood,Longitude,Latitude
0,Barking and Dagenham,0.133543,51.545269
1,Barnet,-0.210019,51.616026
2,Bexley,0.140366,51.458824
3,Brent,-0.267835,51.558553
4,Bromley,0.051530,51.372009
5,Camden,-0.157417,51.546388
6,City of London,-0.092151,51.514827
7,Ealing,-0.331034,51.522471
8,Enfield,-0.087283,51.650994
9,Greenwich,0.056246,51.472810


In [107]:
print('The dataframe has {} neighborhoods.'.format(
        len(london['Neighborhood'].unique()),
        london.shape[0]
    )
)

The dataframe has 33 neighborhoods.


### Creating a map of London

In [108]:
address = 'London'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of London City are {}, {}.'.format(latitude, longitude))

The geographical coordinates of London City are 51.5073219, -0.1276474.


In [109]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(london['Latitude'], london['Longitude'], london['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)

In [110]:
map_london

Getting Foursquare API data

In [111]:
CLIENT_ID = 'EAIAVAVCZBEKDOZAGA5U4HTPMHZJTL5U4ZUAYVEI0FY5XG4F' # your Foursquare ID
CLIENT_SECRET = 'NPPMCRUC5HPH124XXSV1RABP5WDDWXWQZTCFCFOSJJTZN3AM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Your credentails:
CLIENT_ID: EAIAVAVCZBEKDOZAGA5U4HTPMHZJTL5U4ZUAYVEI0FY5XG4F
CLIENT_SECRET:NPPMCRUC5HPH124XXSV1RABP5WDDWXWQZTCFCFOSJJTZN3AM


In [112]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [113]:
london_venues = getNearbyVenues(names = london['Neighborhood'],
                                   latitudes = london['Latitude'],
                                   longitudes = london['Longitude']
                                  )

Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
City of London
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster
Croydon


#### We can see that no venues are present in Bromley

In [114]:
london_venues['Neighborhood'].unique()

array(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Camden',
       'City of London', 'Ealing', 'Enfield', 'Greenwich', 'Hackney',
       'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering',
       'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea',
       'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham',
       'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton',
       'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster',
       'Croydon'], dtype=object)

In [115]:
london_venues['Neighborhood'].nunique()

32

In [116]:
london.drop(4, axis = 0, inplace = True)

#### Neighborhoods without enough venues will also be filtered. This threshold is set to 10 venues minimum.

In [117]:
lots_venues = london_venues.groupby('Neighborhood').count()
lots_venues.reset_index(inplace = True)
lots_venues = lots_venues[lots_venues['Venue'] > 10]

lots_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Bexley,24,24,24,24,24,24
4,Camden,32,32,32,32,32,32
5,City of London,100,100,100,100,100,100
8,Enfield,68,68,68,68,68,68
10,Hackney,21,21,21,21,21,21
11,Hammersmith and Fulham,72,72,72,72,72,72
12,Haringey,39,39,39,39,39,39
14,Havering,24,24,24,24,24,24
16,Hounslow,51,51,51,51,51,51
17,Islington,46,46,46,46,46,46


In [118]:
london = london[london['Neighborhood'].isin(lots_venues['Neighborhood'])]
london

,Neighborhood,Longitude,Latitude
2,Bexley,0.140366,51.458824
5,Camden,-0.157417,51.546388
6,City of London,-0.092151,51.514827
8,Enfield,-0.087283,51.650994
10,Hackney,-0.063334,51.552271
11,Hammersmith and Fulham,-0.221300,51.495960
12,Haringey,-0.107490,51.590369
14,Havering,0.221104,51.564395
16,Hounslow,-0.367074,51.468382
17,Islington,-0.110250,51.548467


In [119]:
print('There are {} uniques categories.'.format(london_venues['Venue Category'].nunique()))

There are 191 uniques categories.


In [120]:
venue_amt = london_venues.groupby('Neighborhood').count()['Venue'].to_frame().reset_index()
venue_amt

,Neighborhood,Venue
0,Barking and Dagenham,2
1,Barnet,2
2,Bexley,24
3,Brent,4
4,Camden,32
5,City of London,100
6,Croydon,2
7,Ealing,4
8,Enfield,68
9,Greenwich,4


### Analyzing each neighborhood's venues

In [121]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head(10)

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Canal Lock,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Locksmith,Lounge,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Optical Shop,Outdoor Supply Store,Park,Pedestrian Plaza,Persian Restaurant,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Recording Studio,Recreation Center,Restaurant,River,Road,Roof Deck,Rugby Pitch,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Soccer Field,Social Club,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Barnet,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Barnet,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

##### We will filter those neighberhoods where there are not enough venues and no hotels

In [122]:
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped = london_grouped[(london_grouped['Hotel'] == 0) & (london_grouped['Hotel Bar'] == 0)]
london_grouped = london_grouped[london_grouped['Neighborhood'].isin(lots_venues['Neighborhood'])]
london_grouped

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Canal Lock,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Locksmith,Lounge,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Optical Shop,Outdoor Supply Store,Park,Pedestrian Plaza,Persian Restaurant,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Recording Studio,Recreation Center,Restaurant,River,Road,Roof Deck,Rugby Pitch,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Soccer Field,Social Club,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
8,Enfield,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.029412,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.014706,0.000000,0.0,0.0,0.058824,0.0,0.000000,0.0,0.000000,0.0,0.088235,0.0,0.000000,0.102941,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.014706,0.029412,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.014706,0.0,0.000000,0.0,0.0,0.000000,0.0,0.014706,0.0,0.014706,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.029412,0.000000,0.000000,0.000000,0.029412,0.014706,0.029412,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.029412,0.0,0.0,0.014706,0.029412,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.029412,0.014706,0.014706,0.0,0.000000,0.029412,0.0,0.000000,0.014706,0.014706,0.000000,0.0,0.0,0.014706,0.058824,0.0,0.000000,0.044118,0.0,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.000000,0.014706,0.029412,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.014706,0.0,0.029412,0.0,0.0,0.0,0.000000,0.014706,0.0,0.000000,0.0,0.014706,0.0,0.000000,0.014706,0.0,0.000000,0.014706,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.014706,0.000000
10,Hackne

### Viewing top venues by neighborhood and creating a dataframe with that data 

In [123]:
num_top_venues = 10

for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Enfield----
              venue  freq
0       Coffee Shop  0.10
1    Clothing Store  0.09
2              Café  0.06
3               Pub  0.06
4        Restaurant  0.04
5         Bookstore  0.03
6          Pharmacy  0.03
7  Department Store  0.03
8     Shopping Mall  0.03
9    Sandwich Place  0.03


----Hackney----
                    venue  freq
0                     Pub  0.14
1           Grocery Store  0.14
2             Yoga Studio  0.10
3          Breakfast Spot  0.10
4              Restaurant  0.10
5                    Café  0.10
6         Coworking Space  0.05
7             Coffee Shop  0.05
8  Furniture / Home Store  0.05
9                    Park  0.05


----Haringey----
                venue  freq
0      Clothing Store  0.10
1            Bus Stop  0.08
2         Coffee Shop  0.08
3            Pharmacy  0.05
4      Sandwich Place  0.05
5  African Restaurant  0.03
6           Multiplex  0.03
7                Café  0.03
8  Chinese Restaurant  0.03
9                Park  0.03



In [124]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [125]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
london_venues_sorted = pd.DataFrame(columns=columns)
london_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    london_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)
    
london_venues_sorted.set_index('Neighborhood')

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
Neighborhood,,,,,,,,,,,,,,,
Enfield,Coffee Shop,Clothing Store,Café,Pub,Restaurant,Optical Shop,Shopping Mall,Sandwich Place,Supermarket,Bookstore,Department Store,Pharmacy,Gift Shop,Gym / Fitness Center,Grocery Store
Hackney,Grocery Store,Pub,Yoga Studio,Breakfast Spot,Café,Restaurant,Gourmet Shop,Park,Furniture / Home Store,Coworking Space,Brewery,Coffee Shop,Italian Restaurant,French Restaurant,Food Truck
Haringey,Clothing Store,Bus Stop,Coffee Shop,Sandwich Place,Pharmacy,Discount Store,Café,Chinese Restaurant,Smoke Shop,Shopping Mall,Pub,Dessert Shop,African Restaurant,Burger Joint,Park
Havering,Italian Restaurant,Grocery Store,Supermarket,Coffee Shop,Pub,Café,Pharmacy,Sandwich Place,Thai Restaurant,Theater,Train Station,Garden,Portuguese Restaurant,Bank,Bakery
Islington,Café,Coffee Shop,Pub,Italian Restaurant,Ethiopian Restaurant,Wine Shop,Vietnamese Restaurant,French Restaurant,Park,Gym / Fitness Center,Farm,Malay Restaurant,Pizza Place,Seafood Restaurant,Yoga Studio
Lambeth,Coffee Shop,Bus Stop,Indian Restaurant,Pub,Music Venue,Grocery Store,Pizza Place,Park,Caribbean Restaurant,Social Club,Spa,Modern European Restaurant,Café,Fried Chicken Joint,Yoga Studio
Lewisham,Grocery Store,Supermarket,Platform,Pub,Coffee Shop,Italian Restaurant,Bus Stop,Greek Restaurant,Dessert Shop,Pharmacy,Portuguese Restaurant,Sandwich Place,Cocktail Bar,Shopping Mall,Shopping Plaza
Newham,Grocery Store,Café,Fish & Chips Shop,Gym / Fitness Center,Pub,Home Service,Convenience Store,Toy / Game Store,French Restaurant,Food Truck,Food Court,Fish Market,Field,Fast Food Restaurant,Yoga Studio
Tower Hamlets,Canal Lock,Pub,Bakery,Chinese Restaurant,Museum,Skate Park,Go Kart Track,Persian Restaurant,Playground,Diner,Recreation Center,Athletics & Sports,Art Gallery,Eastern European Restaurant,Food Truck


### Clustering neighborhoods

In [126]:
# set number of clusters
kclusters = 3

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 2, 0], dtype=int32)

In [127]:
#london_venues_sorted.drop('Cluster Labels', axis = 1, inplace = True)
london_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = london[london['Neighborhood'].isin(london_grouped['Neighborhood'])]

In [128]:
london_merged = london_merged.join(london_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [129]:
london_merged

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
8,Enfield,-0.087283,51.650994,1,Coffee Shop,Clothing Store,Café,Pub,Restaurant,Optical Shop,Shopping Mall,Sandwich Place,Supermarket,Bookstore,Department Store,Pharmacy,Gift Shop,Gym / Fitness Center,Grocery Store
10,Hackney,-0.063334,51.552271,1,Grocery Store,Pub,Yoga Studio,Breakfast Spot,Café,Restaurant,Gourmet Shop,Park,Furniture / Home Store,Coworking Space,Brewery,Coffee Shop,Italian Restaurant,French Restaurant,Food Truck
12,Haringey,-0.107490,51.590369,1,Clothing Store,Bus Stop,Coffee Shop,Sandwich Place,Pharmacy,Discount Store,Café,Chinese Restaurant,Smoke Shop,Shopping Mall,Pub,Dessert Shop,African Restaurant,Burger Joint,Park
14,Havering,0.221104,51.564395,1,Italian Restaurant,Grocery Store,Supermarket,Coffee Shop,Pub,Café,Pharmacy,Sandwich Place,Thai Restaurant,Theater,Train Station,Garden,Portuguese Restaurant,Bank,Bakery
17,Islington,-0.110250,51.548467,1,Café,Coffee Shop,Pub,Italian Restaurant,Ethiopian Restaurant,Wine Shop,Vietnamese Restaurant,French Restaurant,Park,Gym / Fitness Center,Farm,Malay Restaurant,Pizza Place,Seafood Restaurant,Yoga Studio
20,Lambeth,-0.118292,51.453084,1,Coffee Shop,Bus Stop,Indian Restaurant,Pub,Music Venue,Grocery Store,Pizza Place,Park,Caribbean Restaurant,Social Club,Spa,Modern European Restaurant,Café,Fried Chicken Joint,Yoga Studio
21,Lewisham,-0.020276,51.448095,1,Grocery Store,Supermarket,Platform,Pub,Coffee Shop,Italian Restaurant,Bus Stop,Greek Restaurant,Dessert Shop,Pharmacy,Portuguese Restaurant,Sandwich Place,Cocktail Bar,Shopping Mall,Shopping Plaza
23,Newham,0.036388,51.528359,2,Grocery Store,Café,Fish & Chips Shop,Gym / Fitness Center,Pub,Home Service,Convenience Store,Toy / Game Store,French Restaurant,Food Truck,Food Court,Fish Market,Field,Fast Food Restaurant,Yoga Studio
28,Tower Hamlets,-0.035579,51.517163,0,Canal Lock,Pub,Bakery,Chinese Restaurant,Museum,Skate Park,Go Kart Track,Persian Restaurant,Playground,Diner,Recreation Center,Athletics & Sports,Art Gallery,Eastern European Restaurant,Food Truck


In [130]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Neighborhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the clusters

#### Cluster 1

In [131]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
28,Tower Hamlets,Canal Lock,Pub,Bakery,Chinese Restaurant,Museum,Skate Park,Go Kart Track,Persian Restaurant,Playground,Diner,Recreation Center,Athletics & Sports,Art Gallery,Eastern European Restaurant,Food Truck


#### Cluster 2

In [132]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
8,Enfield,Coffee Shop,Clothing Store,Café,Pub,Restaurant,Optical Shop,Shopping Mall,Sandwich Place,Supermarket,Bookstore,Department Store,Pharmacy,Gift Shop,Gym / Fitness Center,Grocery Store
10,Hackney,Grocery Store,Pub,Yoga Studio,Breakfast Spot,Café,Restaurant,Gourmet Shop,Park,Furniture / Home Store,Coworking Space,Brewery,Coffee Shop,Italian Restaurant,French Restaurant,Food Truck
12,Haringey,Clothing Store,Bus Stop,Coffee Shop,Sandwich Place,Pharmacy,Discount Store,Café,Chinese Restaurant,Smoke Shop,Shopping Mall,Pub,Dessert Shop,African Restaurant,Burger Joint,Park
14,Havering,Italian Restaurant,Grocery Store,Supermarket,Coffee Shop,Pub,Café,Pharmacy,Sandwich Place,Thai Restaurant,Theater,Train Station,Garden,Portuguese Restaurant,Bank,Bakery
17,Islington,Café,Coffee Shop,Pub,Italian Restaurant,Ethiopian Restaurant,Wine Shop,Vietnamese Restaurant,French Restaurant,Park,Gym / Fitness Center,Farm,Malay Restaurant,Pizza Place,Seafood Restaurant,Yoga Studio
20,Lambeth,Coffee Shop,Bus Stop,Indian Restaurant,Pub,Music Venue,Grocery Store,Pizza Place,Park,Caribbean Restaurant,Social Club,Spa,Modern European Restaurant,Café,Fried Chicken Joint,Yoga Studio
21,Lewisham,Grocery Store,Supermarket,Platform,Pub,Coffee Shop,Italian Restaurant,Bus Stop,Greek Restaurant,Dessert Shop,Pharmacy,Portuguese Restaurant,Sandwich Place,Cocktail Bar,Shopping Mall,Shopping Plaza


#### Cluster 3

In [133]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
23,Newham,Grocery Store,Café,Fish & Chips Shop,Gym / Fitness Center,Pub,Home Service,Convenience Store,Toy / Game Store,French Restaurant,Food Truck,Food Court,Fish Market,Field,Fast Food Restaurant,Yoga Studio


# Conclusion

Considering the neighborhhods in London City that have enough venues to be considered as touristic Atractions, it is my belief that Tower Hamlets is the best option as the location of the hotel.

It has museums as the 5th most common venue and art galleries as the 13th, something no other neighberhood has in its top 15. Moreover, pubs are the 2nd most common venues in Tower Hamlets. England beign known for their pubs, it is likely that tourist would like to visit them in a couple of occasions. In addition, several foreign restaurants are present in Tower Hamlets, an attractive idea for dinner to most turists. Finally, it is the second closest neighberhood to the airport in those that we consider viable options for the hotel placement.

All in all, I believe that Tower Hamlets is a perfectly good option for the location of a hotel.